In [1]:
from collections import *
from functools import partial
import numpy as np
import pandas as pd
from pprint import PrettyPrinter
from z3 import *

print = PrettyPrinter().pprint

In [18]:
# persons is a list of ids (for simplicity)
# we can be more sophisticated and use a 
# Person class to represent each person
persons = range(10)
person_ids = range(len(persons))

# tables is a list of table sizes
# e.g. if tables = [3, 4, 5], it means
# there are 3 tables of sizes 3, 4 and 5
# respectively.
tables = [5] * 3
table_ids = range(len(tables))

# groups is a 2d list where groups[i]
# contains all the people in the group
# numbered i.
# e.g. if groups[0] == [1, 2], it means
# person 1 and 2 belong to group 0
groups = [
    [0, 1, 2, 3],
    [6, 7, 8, 9]
]

# friends is 2d list where friends[i] 
# contains all the friends of person i
# e.g. if friends[0] == [1, 2, 3], it means 
# that person 0 has three friends: persons 1, 2, 3
friends = [
    [1],
    [2, 3, 4]
]

# enemies is a list of pairs where
# each pair holds two people that 
# shouldn't be seated on the same table
enemies = []

# couples is a list of pairs where
# each pair holds two people that
# must be seated on the same table
couples = []

In [19]:
def get_assignment_booleans():
    '''
    get_assignment_booleans returns a 2d list assg,
    where assg[i][j] represents the boolean that
    will be true if person i sits on table j. And false
    if person i does not sit on table j.
    '''
    
    def get_person_table_boolean(person_id, table_id):
        return Bool('{}_{}'.format(person_id, table_id))
    
    def get_person_assignments(person_id):
        return [get_person_table_boolean(person_id, table_id) for table_id in table_ids]
        
    return [get_person_assignments(person_id) for person_id in person_ids]
    
assg = get_assignment_booleans()

In [20]:
def get_filtered_assignments(p_ids=person_ids, t_ids=table_ids):
    '''
    get_filtered_assignments takes a list of persons `p_ids`
    and a list of tables `t_ids` and returns a list of booleans
    that correspond to the persons and tables provided. that is,
    it returns a cartesian product of the two lists.
    
    if no args are provided, p_ids and t_ids default to the all
    the persons and all the tables.
    
    for e.g. if p_ids == [10], then it returns all the booleans
    that correspond to person_id == 10, which are all the booleans
    for person 0 and all the tables that person #10 can possibly 
    sit at. hence it returns a boolean list: 
    [10_0, 10_1, 10_2, ..., 10_n] where n is the last table_id.
    
    another example: if p_ids == [2, 5] and t_ids == [3, 4], then
    the resulting list of booleans should be [2_3, 2_4, 5_3, 5_4].
    hence, all the booleans returned are the cartesian product of 
    the p_ids and t_ids provided.
    
    if p_ids and t_ids are both not provided, i.e. they both
    have the default value of all persons and all tables, then
    all possible booleans are returned. i.e.
    [0_0, 0_1, ..., 0_n, 1_0, 1_1, ..., 1_n, m_0, m_1, ..., m_n]
    where m is the last person_id and n is table_id.
    '''
    f_assgs = [assg[p][t] for p in p_ids for t in t_ids]
    return f_assgs

In [21]:
def get_assignment_results(model):
    '''
    get_assignment_results takes in a z3 model
    and returns a 2d list: results where 
    results[i][j] is True if person i sits
    at table j. And False if person i does
    not sit at table j.
    
    this result matrix is a realization of the 
    booleans that the solver solves.
    '''
    if not model:
        return [[]]
    
    results = [[is_true(model.eval(assg[p_i][t_i])) for t_i in table_ids] for p_i in person_ids]
    return results

def get_couple_constraint(p1_id, p2_id):
    c1a = get_filtered_assignments(p_ids=[p1_id])
    c2a = get_filtered_assignments(p_ids=[p2_id])
    return Or(*map(And, (zip(c1a, c2a))))

def get_enemy_constraint(p1_id, p2_id):
    return Not(get_couple_constraint(p1_id, p2_id))

In [22]:
# hard_cons is a list of constraints
# that must be satisfied
hard_cons = []

# soft_cons is a list of constraints
# that need not be satisfied.
# constraints are added to soft_cons
# in the form of (constraint, weight).
# we are not strict about solving all 
# these constraints and are okay if any
# number of them can be solved. ideally
# we want to solve as many of the 
# constraints in soft_cons as possible.
#
# hence, if the solver solves the problem
# quickly, we can add soft_cons to the solver
# and get a solution quickly. else if the client
# is okay waiting for a good solution, we can
# add the soft_cons to the solver and solve 
# as well.
soft_cons = []

# each person can only be in one table
for p_i, _ in enumerate(persons):
    p_assg = get_filtered_assignments(p_ids=[p_i])
    weighted_p_assg = [(p, 1) for p in p_assg]
    hard_cons.append(PbEq(weighted_p_assg, 1))
    
# each table has a fixed capacity
for t_i, t_cap in enumerate(tables):
    t_assg = get_filtered_assignments(t_ids=[t_i])
    weighted_t_assg = [(t, 1) for t in t_assg]
    hard_cons.append(PbLe(weighted_t_assg, t_cap))
    
# couples must be seated together
for c1, c2 in couples:
    hard_cons.append(get_couple_constraint(c1, c2))

# enemies must be seated separately
for e1, e2 in enemies:
    hard_cons.append(get_enemy_constraint(e1, e2))

# Friends should be seated together.
# If x only has one friend y, x should
# be seated with y. If x has more than
# one friend, x should be seated with
# at least two friends.
for p, p_friends in enumerate(friends):
    if len(p_friends) > 1:
        for t in table_ids:
            friend_bools = [(assg[f][t], 1) for f in p_friends]
            ge_2_friends = PbGe(friend_bools, 2)
            hard_cons.append(Implies(assg[p][t], ge_2_friends))
    elif len(p_friends) > 0:
        hard_cons.append(get_couple_constraint(p, p_friends[0]))

# each table should not have more than 70% from
# any single group
for t, t_cap in enumerate(tables):
    group_cap = int(t_cap * 0.7)
    affected_groups = filter(lambda g: len(g) > group_cap, groups)
    for g in affected_groups:
        g_assg = get_filtered_assignments(p_ids=g, t_ids=[t])
        weighted_g_assg = zip(g_assg, [1] * len(g_assg))
        hard_cons.append(PbLe(weighted_g_assg, group_cap))
    
# tables should be as full as possible
# this is a difficult problem, hence we 
# add it to soft_cons
for t in table_ids:
    t_sizes = []
    t_poss = get_filtered_assignments(t_ids=[t])
    x = [PbGe(map(lambda x: (x, 1), t_poss), i) for i in range(1, tables[t] + 1)]
    soft_cons.extend(zip(x, range(1, tables[t] + 1)))

In [23]:
s = Solver()
s.add(And(hard_cons))
%time check_res = s.check()
if check_res == sat:
    results = get_assignment_results(s.model())
    print(pd.DataFrame(results, columns=['table_{}'.format(t) for t in table_ids]))
    ppl_assg = [r.index(True) for r in results]
    print(pd.DataFrame(ppl_assg, columns=['table_id']))
    print(Counter(ppl_assg))
else:
    print('not satisfiable')

CPU times: user 9.44 ms, sys: 0 ns, total: 9.44 ms
Wall time: 10.8 ms
   table_0  table_1  table_2
0     True    False    False
1     True    False    False
2     True    False    False
3    False     True    False
4     True    False    False
5    False     True    False
6    False     True    False
7    False     True    False
8    False     True    False
9     True    False    False
   table_id
0         0
1         0
2         0
3         1
4         0
5         1
6         1
7         1
8         1
9         0
Counter({0: 5, 1: 5})
